<a href="https://colab.research.google.com/github/SiddharthChavan23/Flipkart_WebDataScrape/blob/main/Flipkart_Web_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [67]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":'B_NuCI'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string



def get_disc_price(soup):
    try:
        disc_price = soup.find("div",attrs={'class':'_30jeq3 _16Jk6d'})
        disc_price_value = disc_price.text
    except AttributeError:
        disc_price_value = ""
    return disc_price_value

def get_og_price(soup):
    try:
      og_price = soup.find("div",attrs={'class':'_3I9_wc _2p6lqe'})
      og_price = og_price.text
    except AttributeError:
      try:
        og_price = soup.find("div",attrs={'class':'_17Rl6L'})
        og_price = og_price.text
      except AttributeError:
        og_price = ""
    return og_price
def get_total_discount(soup):
    try:
        get_total_discount = soup.find("div",attrs={'class':'_3Ay6Sb _31Dcoz'})
        get_total_discount = get_total_discount.text
    except AttributeError:
        get_total_discount = "No discount"

    return get_total_discount

In [69]:
if __name__ == '__main__':

  #Header for Requests
  # https://explore.whatismybrowser.com/useragents/parse/
  HEADERS = ({'','Accept-Language':'en-US, en:q=0.5'})

  URL = "https://www.flipkart.com/search?q=television&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_4_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_4_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=television%7CTelevisions&requestId=5b542260-5b8a-4605-b220-3eaae0c7cd64"

  #HTTPS Requests
  webpage = requests.get(URL,headers=HEADERS)
  #Soup Object  containing all data
  soup = BeautifulSoup(webpage.content, "html.parser")
  #Fetch links as List of Tag Object
  links = soup.find_all("a",attrs = {'class':'_1fQZEK'})
  #Store links in a list
  link_lists = []


  for link in links:
    link_lists.append(link.get('href'))

  d = {"title":[],"discounted_price":[],"original_price":[],"total_discount":[]}




  # Loop for extracting each information for each links.
  for link in link_lists:
    new_webpage = requests.get("https://www.flipkart.com" + link,headers = HEADERS)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")


    d['title'].append(get_title(new_soup))
    d['discounted_price'].append(get_disc_price(new_soup))
    d['original_price'].append(get_og_price(new_soup))
    d['total_discount'].append(get_total_discount(new_soup))


  amazon_df = pd.DataFrame.from_dict(d)
  amazon_df['title'].replace('', np.nan, inplace=True)
  amazon_df = amazon_df.dropna(subset=['title'])
  amazon_df.to_csv("amazon_data.csv", header=True, index=False)